In [6]:
%run "Preprocessing.ipynb"  # Run Preprocessing notebook

Preprocessing file executed successfully.


In [7]:
# Call your functions
df_features, df_sales, df_stores = read_csv_files()
df_features, df_sales = process_dates(df_features, df_sales)
df_merged = merge_datasets(df_sales, df_features, df_stores)
df_filled = fill_missing_values(df_merged)
df_extracted = extract_date_components(df_filled)
df_final, dropped_columns = preprocessing()

# Add ID column to df_final and dropped_columns
df_final_with_id = add_id_column(df_final)
dropped_columns_with_id = add_id_column(dropped_columns)




In [8]:
import pandas as pd
import pyodbc

def connect_to_sql_server(server, database, username, password):
    try:
        # Connect to SQL Server using pyodbc
        conn = pyodbc.connect(
            f'DRIVER={{ODBC Driver 17 for SQL Server}};'
            f'SERVER={server};'
            f'DATABASE={database};'
            f'UID={username};'
            f'PWD={password}'
        )
        return conn
    except Exception as e:
        print(f"Error connecting to SQL Server: {e}")
        return None

def create_tables_and_load_data(server, database, username, password, df_merged, df_extracted, df_final):
    try:
        # Connect to SQL Server
        conn = connect_to_sql_server(server, database, username, password)
        if conn is None:
            return
        
        # Create a cursor object using the connection
        cursor = conn.cursor()

        # Define table names and their corresponding DataFrames
        tables = {
            'df_merged': df_merged,
            'df_extracted': df_extracted,
            'AfterPreprocessing': df_final
        }

        # Iterate over tables to create them and insert data
        for table_name, df in tables.items():
            # Drop table if exists
            cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};")
            conn.commit()

            # Create SQL CREATE TABLE statement dynamically
            create_table_sql = f"CREATE TABLE {table_name} ("

            for column in df.columns:
                sql_type = "NVARCHAR(MAX)"  # Default type for non-numeric columns
                # Infer SQL types based on DataFrame dtypes
                if pd.api.types.is_integer_dtype(df[column]):
                    sql_type = "INT"
                elif pd.api.types.is_float_dtype(df[column]):
                    sql_type = "FLOAT"
                elif pd.api.types.is_bool_dtype(df[column]):
                    sql_type = "BIT"
                elif pd.api.types.is_datetime64_any_dtype(df[column]):
                    sql_type = "DATETIME"
                elif pd.api.types.is_string_dtype(df[column]):
                    max_length = df[column].str.len().max()
                    sql_type = f"NVARCHAR({max_length})"

                create_table_sql += f"{column} {sql_type}, "

            create_table_sql = create_table_sql.rstrip(', ') + ");"

            # Execute the CREATE TABLE statement
            cursor.execute(create_table_sql)
            conn.commit()

            # Insert data into the table
            for index, row in df.iterrows():
                data_values = []
                for value in row:
                    if pd.isnull(value):
                        data_values.append(None)
                    elif isinstance(value, (int, float)):
                        data_values.append(value)
                    else:
                        data_values.append(str(value))  # Convert to string if not numeric or null

                # Execute the INSERT statement with the prepared data values
                insert_sql = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['?'] * len(df.columns))})"
                cursor.execute(insert_sql, tuple(data_values))
                conn.commit()

            print(f'Table "{table_name}" created and data inserted successfully.')

        # Close the cursor and connection
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error: {e}")

    
    # Call the function to create tables and load data
    #create_tables_and_load_data(server, database, username, password, df_merged, df_extracted, df_final)


In [9]:
def create_predictions_table_and_load_data(server, database, username, password, df_predictions, table_name_predictions):
    try:
        # Connect to SQL Server
        conn = connect_to_sql_server(server, database, username, password)
        if conn is None:
            return
        
        # Create a cursor object using the connection
        cursor = conn.cursor()

        # Drop the table if it exists
        cursor.execute(f"IF OBJECT_ID('{table_name_predictions}', 'U') IS NOT NULL DROP TABLE {table_name_predictions};")
        conn.commit()

        # Create SQL CREATE TABLE statement dynamically
        create_table_sql = f"CREATE TABLE {table_name_predictions} ("

        for column in df_predictions.columns:
            sql_type = "NVARCHAR(MAX)"  # Default type for non-numeric columns
            # Infer SQL types based on DataFrame dtypes
            if pd.api.types.is_integer_dtype(df_predictions[column]):
                sql_type = "INT"
            elif pd.api.types.is_float_dtype(df_predictions[column]):
                sql_type = "FLOAT"
            elif pd.api.types.is_bool_dtype(df_predictions[column]):
                sql_type = "BIT"
            elif pd.api.types.is_datetime64_any_dtype(df_predictions[column]):
                sql_type = "DATETIME"
            elif pd.api.types.is_string_dtype(df_predictions[column]):
                max_length = df_predictions[column].str.len().max()
                sql_type = f"NVARCHAR({max_length})"

            create_table_sql += f"{column} {sql_type}, "

        create_table_sql = create_table_sql.rstrip(', ') + ");"

        # Execute the CREATE TABLE statement
        cursor.execute(create_table_sql)
        conn.commit()

        # Insert data into the predictions table
        for index, row in df_predictions.iterrows():
            data_values = []
            for value in row:
                if pd.isnull(value):
                    data_values.append(None)
                elif isinstance(value, (int, float)):
                    data_values.append(value)
                else:
                    data_values.append(str(value))  # Convert to string if not numeric or null

            # Execute the INSERT statement with the prepared data values
            insert_sql = f"INSERT INTO {table_name_predictions} ({', '.join(df_predictions.columns)}) VALUES ({', '.join(['?'] * len(df_predictions.columns))})"
            cursor.execute(insert_sql, tuple(data_values))
            conn.commit()

        print(f'Table "{table_name_predictions}" created and data inserted successfully.')

        # Close the cursor and connection
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error: {e}")

In [10]:
print('SQLServerLink file executed successfully.')

SQLServerLink file executed successfully.
